# Baseline Linear Regression

In [3]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# two levels up from the notebook to project root, then append to sys.path
sys.path.append(str(Path().resolve().parents[1]))

PROJECT_ROOT = Path().resolve().parents[1]
RESULTS_PATH = PROJECT_ROOT / "results"


from utils.data_loading import load_datasets
from utils.data_preparation import get_spo2_to_po2_interpolator, add_shift_raw_column, add_engineered_features
from utils.evaluation import evaluate_macro_patient_level, print_evaluation, bland_altman_plots
from utils.modeling import fit_cv_models, predict_cv_ensemble
from utils.logging import log_run_json


train_df, test_df, val_df, odc = load_datasets()
spo2_to_po2 = get_spo2_to_po2_interpolator(odc)

train_df = add_shift_raw_column(train_df, spo2_to_po2)
test_df  = add_shift_raw_column(test_df, spo2_to_po2)
val_df   = add_shift_raw_column(val_df, spo2_to_po2)

# Effect of Difference of Feature Sets

## With Fixed Shift_raw

In [4]:
import itertools
import pandas as pd
from sklearn.linear_model import LinearRegression



train_df_engineered = add_engineered_features(train_df, spo2_to_po2)
test_df_engineered = add_engineered_features(test_df, spo2_to_po2)

# Always include 'shift_raw'
base_features = ['PiO2(kPa)', 'SpO2(%)', 'Hb', 'log_PiO2', 'log_SpO2', 'SpO2_over_PiO2', 
                 'SpO2_squared', 'Hb_SpO2', 'saturation_deficit', 'CaO2_estimate']

mandatory_feature = ['shift_raw']

# Generate all combinations of additional features (1 to N), always adding shift_raw
feature_combinations = [
    mandatory_feature + list(combo)
    for r in range(1, len(base_features) + 1)
    for combo in itertools.combinations(base_features, r)
]


# Evaluate each feature set
results = []

for features in feature_combinations:
    features = list(features)
    model = LinearRegression()
    cv_models = fit_cv_models(train_df_engineered, features=features, target_col='shift', model_class=LinearRegression, k=10)
    test_df_engineered['y_pred'] = predict_cv_ensemble(test_df_engineered, features, cv_models)

    summary = evaluate_macro_patient_level(test_df_engineered, y_true_col='shift', y_pred_col='y_pred')
    results.append({
        'features': features,
        'MAE': summary['MAE'],
        'MSE': summary['MSE'],
        'RMSE': summary['RMSE'],
        'Bias': summary['Mean Bias Error']
    })

# Convert results to DataFrame for inspection
results_df = pd.DataFrame(results).sort_values(by='MAE')


In [5]:
results_df

,features,MAE,MSE,RMSE,Bias
281,"[shift_raw, SpO2(%), log_PiO2, log_SpO2, SpO2_...",0.890621,1.733317,1.081507,0.024642
551,"[shift_raw, SpO2(%), log_PiO2, log_SpO2, SpO2_...",0.890621,1.733317,1.081507,0.024642
355,"[shift_raw, log_PiO2, log_SpO2, SpO2_squared, ...",0.890621,1.733317,1.081507,0.024642
150,"[shift_raw, log_PiO2, SpO2_squared, saturation...",0.891493,1.736509,1.082282,0.024563
100,"[shift_raw, SpO2(%), log_PiO2, SpO2_squared]",0.891493,1.736509,1.082282,0.024563
...,...,...,...,...,...
16,"[shift_raw, PiO2(kPa), Hb_SpO2]",1.123795,3.710883,1.312705,-0.074786
18,"[shift_raw, PiO2(kPa), CaO2_estimate]",1.123795,3.710883,1.312705,-0.074786
89,"[shift_raw, PiO2(kPa), Hb_SpO2, CaO2_estimate]",1.123795,3.710883,1.312705,-0.074786
53,"[shift_raw, Hb_SpO2, CaO2_estimate]",1.123795,3.710883,1.312705,-0.074786


In [6]:
results_df_rounded = results_df.copy()

for col in results_df.columns:
    if col != 'features':
        results_df_rounded[col] = pd.to_numeric(results_df[col], errors='coerce').round(3)
results_df_rounded.head(10).to_csv(RESULTS_PATH / "linear_regression_subsets.csv", index=False)

In [7]:
top_n = 10
top_features = results_df.nsmallest(top_n, 'MAE')['features']

from collections import Counter
flat_features = [f for sublist in top_features for f in sublist]
freq_counter = Counter(flat_features)

# View most frequent features in top-N models
sorted(freq_counter.items(), key=lambda x: x[1], reverse=True)


[('shift_raw', 10),
 ('log_PiO2', 7),
 ('log_SpO2', 7),
 ('SpO2_squared', 7),
 ('SpO2(%)', 6),
 ('saturation_deficit', 6),
 ('SpO2_over_PiO2', 3)]

# True Stepwise-Forward Selection

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd



train_df_engineered = add_engineered_features(train_df, spo2_to_po2)
test_df_engineered = add_engineered_features(test_df, spo2_to_po2)

def adjusted_r2_score(y_true, y_pred, n, p):
    r2 = r2_score(y_true, y_pred)
    return 1 - ((1 - r2) * (n - 1) / (n - p - 1))

# Base features
base_features = ['PiO2(kPa)', 'SpO2(%)', 'Hb', 'log_PiO2', 'log_SpO2',
                 'SpO2_over_PiO2', 'SpO2_squared', 'Hb_SpO2',
                 'saturation_deficit', 'CaO2_estimate']

selected_features = ['shift_raw']
remaining_features = base_features.copy()

improvement = True
results = []

while improvement and remaining_features:
    best_feature = None
    best_adj_r2 = -np.inf

    for feature in remaining_features:
        candidate_features = selected_features + [feature]
        X = train_df_engineered[candidate_features]
        y = train_df_engineered['shift']

        model = LinearRegression().fit(X, y)
        y_pred = model.predict(X)
        adj_r2 = adjusted_r2_score(y, y_pred, n=len(y), p=len(candidate_features))

        if adj_r2 > best_adj_r2:
            best_adj_r2 = adj_r2
            best_feature = feature

    current_X = train_df_engineered[selected_features]
    current_model = LinearRegression().fit(current_X, train_df_engineered['shift'])
    current_pred = current_model.predict(current_X)
    current_adj_r2 = adjusted_r2_score(train_df_engineered['shift'], current_pred, len(train_df_engineered), len(selected_features))

    if best_adj_r2 > current_adj_r2:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        results.append((selected_features.copy(), best_adj_r2))
    else:
        improvement = False

# Final selected features and their adjusted R²
for idx, (features, score) in enumerate(results):
    print(f"Step {idx+1}: {features} → Adjusted R² = {score:.4f}")


Step 1: ['shift_raw', 'SpO2_over_PiO2'] → Adjusted R² = 0.8355
Step 2: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared'] → Adjusted R² = 0.8405
Step 3: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared', 'log_PiO2'] → Adjusted R² = 0.8657
Step 4: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared', 'log_PiO2', 'saturation_deficit'] → Adjusted R² = 0.8719
Step 5: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared', 'log_PiO2', 'saturation_deficit', 'PiO2(kPa)'] → Adjusted R² = 0.8737
Step 6: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared', 'log_PiO2', 'saturation_deficit', 'PiO2(kPa)', 'log_SpO2'] → Adjusted R² = 0.8744
Step 7: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared', 'log_PiO2', 'saturation_deficit', 'PiO2(kPa)', 'log_SpO2', 'Hb_SpO2'] → Adjusted R² = 0.8749
Step 8: ['shift_raw', 'SpO2_over_PiO2', 'SpO2_squared', 'log_PiO2', 'saturation_deficit', 'PiO2(kPa)', 'log_SpO2', 'Hb_SpO2', 'Hb'] → Adjusted R² = 0.8753


In [9]:
import pandas as pd

# Prepare results for export
results_table = pd.DataFrame([
    {
        "Step": idx + 1,
        "Num_Features": len(features),
        "Features": ", ".join(features),
        "Adjusted_R2": round(score, 3)
    }
    for idx, (features, score) in enumerate(results)
])

# Export to CSV in the results folder
results_table.to_csv(RESULTS_PATH / "linear_regression_stepwise_forward_selection_results.csv", index=False)
results_table

,Step,Num_Features,Features,Adjusted_R2
0,1,2,"shift_raw, SpO2_over_PiO2",0.836
1,2,3,"shift_raw, SpO2_over_PiO2, SpO2_squared",0.840
2,3,4,"shift_raw, SpO2_over_PiO2, SpO2_squared, log_PiO2",0.866
3,4,5,"shift_raw, SpO2_over_PiO2, SpO2_squared, log_P...",0.872
4,5,6,"shift_raw, SpO2_over_PiO2, SpO2_squared, log_P...",0.874
5,6,7,"shift_raw, SpO2_over_PiO2, SpO2_squared, log_P...",0.874
6,7,8,"shift_raw, SpO2_over_PiO2, SpO2_squared, log_P...",0.875
7,8,9,"shift_raw, SpO2_over_PiO2, SpO2_squared, log_P...",0.875


## Model with True Stepwise Forward Selection

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# two levels up from the notebook to project root, then append to sys.path
sys.path.append(str(Path().resolve().parents[1]))

PROJECT_ROOT = Path().resolve().parents[1]
RESULTS_PATH = PROJECT_ROOT / "results"


from utils.data_loading import load_datasets
from utils.data_preparation import get_spo2_to_po2_interpolator, add_shift_raw_column, add_engineered_features
from utils.evaluation import evaluate_macro_patient_level, print_evaluation, bland_altman_plots, bland_altman_pct_comparison
from utils.modeling import fit_cv_models, predict_cv_ensemble
from utils.logging import log_run_json


train_df, test_df, val_df, odc = load_datasets()
spo2_to_po2 = get_spo2_to_po2_interpolator(odc)

train_df = add_shift_raw_column(train_df, spo2_to_po2)
test_df  = add_shift_raw_column(test_df, spo2_to_po2)
val_df   = add_shift_raw_column(val_df, spo2_to_po2)

In [2]:
from sklearn.linear_model import LinearRegression

In [4]:
train_df_engineered = add_engineered_features(train_df, spo2_to_po2)
test_df_engineered = add_engineered_features(test_df, spo2_to_po2)
val_df_engineered = add_engineered_features(val_df, spo2_to_po2)

features = ['shift_raw', 'SpO2_over_PiO2',  'SpO2_squared', 'log_PiO2']

# Train ensemble of CV models
cv_models = fit_cv_models(
    train_df,
    features=features,
    target_col='shift',
    model_class=LinearRegression,
    k=10
)

# Predict on test set (average over CV models)
test_df['y_pred'] = predict_cv_ensemble(test_df, features, cv_models)

# Evaluate macro-averaged patient metrics
test_summary = evaluate_macro_patient_level(test_df, y_true_col='shift', y_pred_col='y_pred')
print_evaluation(test_summary)


Macro-averaged per-patient metrics:
MAE  = 0.972
MSE  = 2.255
RMSE = 1.157
Mean Bias Error = 0.017
MAPE = 9.190%
nRMSE = 6.150%


In [5]:
val_df['y_pred'] = predict_cv_ensemble(val_df, features, cv_models)

val_summary = evaluate_macro_patient_level(val_df, y_true_col='shift', y_pred_col='y_pred')
print_evaluation(val_summary)


Macro-averaged per-patient metrics:
MAE  = 1.140
MSE  = 3.377
RMSE = 1.417
Mean Bias Error = 0.243
MAPE = 9.428%
nRMSE = 7.268%


In [6]:
description = f'''
Model: Linear Regression Optimal Stepwise Forward Selection
Description: This is a model that is trained on the optimal features derived from the forward stepwise selection process.
Features: shift_raw, log_PiO2, SpO2_squared, saturation_deficit
Target: shift
Notes: patient-level macro metrics, ODC from neonatal table
'''

# Filter only scalar (JSON-serializable) entries
json_test_metrics = {
    k: float(v) if isinstance(v, (np.generic, np.float64, np.float32)) else v
    for k, v in test_summary.items()
    if not isinstance(v, pd.Series)
}
json_val_metrics = {
    k: float(v) if isinstance(v, (np.generic, np.float64, np.float32)) else v
    for k, v in val_summary.items()
    if not isinstance(v, pd.Series)
}

json_path = RESULTS_PATH / "single_point_model_metrics_log.json"

log_run_json(
    identifier="Linear Regression Optimal features",
    model_type="Linear Regression",
    features=features,
    train_subset="full train set",
    test_subset="full test set",
    val_subset="full validation set",  
    description=description,
    test_metrics=json_test_metrics,
    val_metrics=json_val_metrics,

    json_path=json_path
)


✅ Logged run #3 ➜ /Users/sarah/Code/neonatal-odc-shift/results/single_point_model_metrics_log.json
